# All pairs shortest path algorithm

Shortest paths between every pair of vertices -algorithm implemented by QUBO in quantum annealer. Classical version of this is Floyd-Warshall algorithm: https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm

Here directed graph $G=(V,E)$ has sets of vertices $V$ and edges $E \subseteq \{(x,y)|(x,y) \in V^2 and x \not = y\}$. For every edge there is weight $w_{xy}$. Task is to find path with minimum sum of weights for every combination of $(s,t) \in E^2$.

In this solution binary variable vector $x$ to be solved has $|V|+|V|+|E|$ elements. First $|V|$ elements indicate the starting vertex and second $|V|$ elements indicate the ending vertex. Last $|E|$ elements would define which edges are part of the shortest path. 

Penalty $p=\sum w_{xy}+1$.

QUBO should implement following constraints to proper $x$: 
1. There must be exactly one starting vertex $s$ (if more than one: penalty of $p$)
2. There must be exactly one terminating vertex $t$ (if more than one: penalty of $p$)
3. Vertices $s$ and $t$ must different (if not: penalty of $p$)
4. There must edge starting from $s$ (gain of $-p$ for each) and no edge terminating to $s$ ($p$ for each)
5. There must edge ending to $t$ (gain of $-p$ for each) and no edge starting from $t$ ($p$ for each)
6. Two edges should not start/end to the same vertex, for example $s$ or $t$ (if so: penalty of $p$) 
7. Two edges should form a chain (two edges forming a chain gives gain of $-p/2$)
8. Path having lower weights should be prioritised (penalty of $w$ to each edge)

Proper path with correct $s$ and $t$ would give energy level of $w-p$ (constraint 7 brings minimum $p$ and constraint 4 and 5 both minimum $-p$). Because $p=\sum w_{xy}$ all samples with energy level below zero are correct paths. From this set of samples we choose lowest energy level sample for each $(s,t)$. 

In [1]:
import numpy as np
import time
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.samplers import SimulatedAnnealingSampler
import dwave.inspector
import networkx as nx
from networkx.classes.function import path_weight
import random

## Function to create QUBO

E is array of tuples (1st vertice, 2nd vertice, weight), veritices is number of vertices, p is penalty.

In [2]:
def create_qubo(E,vertices,p):
    edges = len(E)
    Q = np.zeros((2*vertices + edges, 2*vertices + edges))

    # Constraints 1 and 2
    for i in range(vertices):
        for j in range(vertices):
            if i!=j:
                Q[i,j] += p
                Q[vertices+i,j+vertices] += p
        
    # Constraint 3
    for i in range(vertices):
        Q[i,i+vertices] += p

    # Constraint 4
    for v in range(vertices):
        for i,e in enumerate(E):
            if e[0]==v:
                Q[v,vertices*2+i] -= p
            if e[1]==v:
                Q[v,vertices*2+i] += p

    # Constraint 5
    for v in range(vertices):
        for i,e in enumerate(E):
            if e[1]==v:
                Q[vertices+v,vertices*2+i] -= p
            if e[0]==v:
                Q[vertices+v,vertices*2+i] += p

    # Constraint 6 and 7
    for i,ei in enumerate(E):
        for j,ej in enumerate(E):
            if ei[0]==ej[0] or ei[1]==ej[1]:
                Q[vertices*2+i,vertices*2+j] += p
            if ei[1]==ej[0] or ei[0]==ej[1]:
                Q[vertices*2+i,vertices*2+j] -= p/2

    # Constraint 8 
    for i in range(edges):
        Q[vertices*2+i,vertices*2+i] += E[i][2]

    # Quadratic coefficients in lower triangle to upper triangle
    for i in range(vertices): 
        for j in range(i):
            Q[j,i] += Q[i,j]
            Q[i,j] = 0
            
    return Q

## Some helper functions for getting results from sampleset

In [3]:
def path_from_sample(sample,E,vertices):
    s = 0
    t = 0
    w = 0
    for v in range(vertices):
        if sample['s'+str(v)]==1:
            s = v
        if sample['t'+str(v)]==1:
            t = v
    i = s
    path = [i]
    while i!=t:
        for e in E:
            if e[0]==i and sample[str(e[0]) + '-' + str(e[1])]==1:
                i = e[1]
                path.append(i)
                w += e[2]
    return (str(s)+'-'+str(t),path,w)
        

def result_info(sampleset, E, vertices):
    res = {}
    for s in sampleset.filter(lambda s: s.energy<0):
        #print(s)
        st, path, w = path_from_sample(s,E,vertices)
        if st not in res:
            res[st]=(path,w)
    return res

def check_result(G,sampleset,E,vertices, verbose=True):
    ok = 0
    s = 0
    res = result_info(sampleset,E,vertices)
    for i in range(vertices):
        for j in range(vertices):
            if i!=j:
                if nx.has_path(G,i,j):
                    s += 1
                    p1 = [p for p in nx.all_shortest_paths(G,i,j,weight='weight')]
                    w = path_weight(G,p1[0],'weight')
                    if str(i)+'-'+str(j) in res.keys():
                        p2 = res[str(i)+'-'+str(j)]
                        if (not p2[0] in p1) and w!=p2[1]:
                            if verbose:
                                print('Path: '+str(p2[0])+' ('+str(p2[1])+'): correct: '+str(p1)+' ('+str(w)+')')
                        else:
                            ok += 1
                    else:
                        if verbose:
                            print('Path suggested: '+str(i)+'-'+str(j)+' missing: correct: '+str(p1)+' ('+str(w)+')')
    print('{:.0f}% ok'.format((100*ok/s)))

def make_G(E,vertices):
    G = nx.DiGraph()
    G.add_nodes_from([0, vertices-1])
    for e in E:
        G.add_edge(e[0], e[1], weight=e[2])
    return(G)

## Simple graph

### Define graph

Input graph is array of tuples (1st vertice, 2nd vertice, weight)

In [4]:
E1 = np.array([(0, 2, 1), (2, 1, 2), (1, 3, 3), (3, 2, 4), (0, 1, 5), (3, 4, 3), (2, 4, 8)])
vertices1 = 5
G1 = make_G(E1,vertices1)

Above graph visualised:

![](graph4.png)

### Max_count and labels

In [5]:
max_count1 = 0
edges = len(E1)
for e in E1:
    max_count1 += e[2]
print('Max count:',max_count1)

labels1 = {}
for i in range(vertices1):
    labels1[i]='s'+str(i)
    labels1[vertices1+i]='t'+str(i)   
for i,e in enumerate(E1):
    labels1[vertices1*2+i] = str(e[0]) + '-' + str(e[1])

Max count: 26


### Create QUBO and BQM

In [6]:
t1 = time.time()
Q1 = create_qubo(E1,vertices1,max_count1+1)
qubo_time = (time.time()-t1)*1000
print('Time used for construction Q (ms): {:.3f}\n'.format(qubo_time))

t1 = time.time()
bqm1 = dimod.BinaryQuadraticModel(Q1, 'BINARY')
bqm_time = (time.time()-t1)*1000
bqm1 = bqm1.relabel_variables(labels1, inplace=False)

Time used for construction Q (ms): 0.391



### Local deterministic solver

In [7]:
t1 = time.time()
sampleset = dimod.ExactSolver().sample(bqm1)
det_time = (time.time()-t1)*1000
print('Time used (ms): {:.3f}\n'.format(det_time))
print(sampleset.filter(lambda s: s.energy<0))

Time used (ms): 201.555

   0-1 0-2 1-3 2-1 2-4 3-2 3-4 s0 s1 s2 s3 s4 t0 t1 t2 t3 t4 energy num_oc.
0    0   1   0   0   0   0   0  1  0  0  0  0  0  0  1  0  0  -26.0       1
2    0   0   0   1   0   0   0  0  0  1  0  0  0  1  0  0  0  -25.0       1
1    0   1   0   1   0   0   0  1  0  0  0  0  0  1  0  0  0  -24.0       1
5    0   0   1   0   0   0   0  0  1  0  0  0  0  0  0  1  0  -24.0       1
16   0   0   0   0   0   0   1  0  0  0  1  0  0  0  0  0  1  -24.0       1
8    0   0   0   0   0   1   0  0  0  0  1  0  0  0  1  0  0  -23.0       1
3    0   0   1   1   0   0   0  0  0  1  0  0  0  0  0  1  0  -22.0       1
11   1   0   0   0   0   0   0  1  0  0  0  0  0  1  0  0  0  -22.0       1
4    0   1   1   1   0   0   0  1  0  0  0  0  0  0  0  1  0  -21.0       1
7    0   0   0   1   0   1   0  0  0  0  1  0  0  1  0  0  0  -21.0       1
13   0   0   1   0   0   0   1  0  1  0  0  0  0  0  0  0  1  -21.0       1
6    0   0   1   0   0   1   0  0  1  0  0  0  0  0  1  0  0  -

In [8]:
for k,v in result_info(sampleset, E1, vertices1).items():
    print('Route '+k+': '+str(v[0])+', weight '+str(v[1]))

Route 0-2: [0, 2], weight 1
Route 2-1: [2, 1], weight 2
Route 0-1: [0, 2, 1], weight 3
Route 1-3: [1, 3], weight 3
Route 3-4: [3, 4], weight 3
Route 3-2: [3, 2], weight 4
Route 2-3: [2, 1, 3], weight 5
Route 0-3: [0, 2, 1, 3], weight 6
Route 3-1: [3, 2, 1], weight 6
Route 1-4: [1, 3, 4], weight 6
Route 1-2: [1, 3, 2], weight 7
Route 2-4: [2, 1, 3, 4], weight 8
Route 0-4: [0, 2, 1, 3, 4], weight 9


In [9]:
check_result(G1,sampleset,E1,vertices1)

100% ok


### Local heuristic classical solver

In [10]:
num_reads = 2000
t1 = time.time()
sampleset2 = SimulatedAnnealingSampler().sample(bqm1, num_reads=num_reads).aggregate()
heur_time = (time.time()-t1)*1000
print('Time used (ms): {:.3f}\n'.format(heur_time))
print(sampleset2.filter(lambda s: s.energy<0))

Time used (ms): 642.410

   0-1 0-2 1-3 2-1 2-4 3-2 3-4 s0 s1 s2 s3 s4 t0 t1 t2 t3 t4 energy num_oc.
0    0   1   0   0   0   0   0  1  0  0  0  0  0  0  1  0  0  -26.0     240
10   0   0   0   1   0   0   0  0  0  1  0  0  0  1  0  0  0  -25.0     206
4    0   1   0   1   0   0   0  1  0  0  0  0  0  1  0  0  0  -24.0     180
13   0   0   1   0   0   0   0  0  1  0  0  0  0  0  0  1  0  -24.0     149
15   0   0   0   0   0   0   1  0  0  0  1  0  0  0  0  0  1  -24.0     179
5    0   0   0   0   0   1   0  0  0  0  1  0  0  0  1  0  0  -23.0     137
3    1   0   0   0   0   0   0  1  0  0  0  0  0  1  0  0  0  -22.0     114
7    0   0   1   1   0   0   0  0  0  1  0  0  0  0  0  1  0  -22.0     113
2    0   0   0   1   0   1   0  0  0  0  1  0  0  1  0  0  0  -21.0      89
9    0   0   1   0   0   0   1  0  1  0  0  0  0  0  0  0  1  -21.0      96
17   0   1   1   1   0   0   0  1  0  0  0  0  0  0  0  1  0  -21.0      90
11   0   0   1   0   0   1   0  0  1  0  0  0  0  0  1  0  0  -

In [11]:
for k,v in result_info(sampleset2,E1,vertices1).items():
    print('Route '+k+': '+str(v[0])+', weight '+str(v[1]))

Route 0-2: [0, 2], weight 1
Route 2-1: [2, 1], weight 2
Route 0-1: [0, 2, 1], weight 3
Route 3-4: [3, 4], weight 3
Route 1-3: [1, 3], weight 3
Route 3-2: [3, 2], weight 4
Route 2-3: [2, 1, 3], weight 5
Route 3-1: [3, 2, 1], weight 6
Route 0-3: [0, 2, 1, 3], weight 6
Route 1-4: [1, 3, 4], weight 6
Route 1-2: [1, 3, 2], weight 7
Route 2-4: [2, 1, 3, 4], weight 8
Route 0-4: [0, 2, 1, 3, 4], weight 9


In [12]:
check_result(G1,sampleset2,E1,vertices1)

100% ok


### Quantum solver

In [16]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [17]:
num_reads = 2000
sampleset3 = EmbeddingComposite(machine).sample(bqm1, num_reads=num_reads).aggregate()

In [18]:
qpu_time = sampleset3.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset3.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(qpu_time))
print('Physical qubits used: {}\n'.format(qubits))
print(sampleset3.filter(lambda s: s.energy<0))

QPU time used (ms): 179.9
Physical qubits used: 32

   0-1 0-2 1-3 2-1 2-4 3-2 3-4 s0 s1 s2 s3 s4 t0 t1 t2 t3 t4 energy num_oc. ...
0    0   1   0   0   0   0   0  1  0  0  0  0  0  0  1  0  0  -26.0     141 ...
1    0   0   0   1   0   0   0  0  0  1  0  0  0  1  0  0  0  -25.0     152 ...
2    0   1   0   1   0   0   0  1  0  0  0  0  0  1  0  0  0  -24.0     129 ...
3    0   0   1   0   0   0   0  0  1  0  0  0  0  0  0  1  0  -24.0     190 ...
4    0   0   0   0   0   0   1  0  0  0  1  0  0  0  0  0  1  -24.0      59 ...
5    0   0   0   0   0   1   0  0  0  0  1  0  0  0  1  0  0  -23.0      72 ...
6    1   0   0   0   0   0   0  1  0  0  0  0  0  1  0  0  0  -22.0      33 ...
7    0   0   1   1   0   0   0  0  0  1  0  0  0  0  0  1  0  -22.0     120 ...
8    0   0   0   1   0   1   0  0  0  0  1  0  0  1  0  0  0  -21.0     136 ...
9    0   1   1   1   0   0   0  1  0  0  0  0  0  0  0  1  0  -21.0     114 ...
10   0   0   1   0   0   0   1  0  1  0  0  0  0  0  0  0  1  -21.0 

In [20]:
print('Number of logical qubits:',Q1.shape[0])
print('Number of couplers:', len(bqm1.quadratic))
qpu_time = sampleset3.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset3.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(qpu_time))
print('Physical qubits used: {}\n'.format(qubits))

Number of logical qubits: 17
Number of couplers: 67
QPU time used (ms): 179.9
Physical qubits used: 32



In [21]:
for k,v in result_info(sampleset3,E1,vertices1).items():
    print('Route '+k+': '+str(v[0])+', weight '+str(v[1]))

Route 0-2: [0, 2], weight 1
Route 2-1: [2, 1], weight 2
Route 0-1: [0, 2, 1], weight 3
Route 1-3: [1, 3], weight 3
Route 3-4: [3, 4], weight 3
Route 3-2: [3, 2], weight 4
Route 2-3: [2, 1, 3], weight 5
Route 0-3: [0, 2, 1, 3], weight 6
Route 1-4: [1, 3, 4], weight 6
Route 3-1: [3, 2, 1], weight 6
Route 1-2: [1, 3, 2], weight 7
Route 2-4: [2, 1, 3, 4], weight 8
Route 0-4: [0, 2, 1, 3, 4], weight 9


In [22]:
check_result(G1,sampleset3,E1,vertices1)

100% ok


### Hybrid solver

In [23]:
sampleset3 = LeapHybridSampler().sample(bqm1)
hyb_time = sampleset3.info['qpu_access_time'] / 1000
run_time = sampleset3.info['run_time'] / 1000
print('QPU time used (ms): {:.1f}'.format(hyb_time))
print('Total time used (ms): {:.1f}\n'.format(run_time))
print(sampleset3) 

QPU time used (ms): 104.9
Total time used (ms): 2995.6

  0-1 0-2 1-3 2-1 2-4 3-2 3-4 s0 s1 s2 s3 s4 t0 t1 t2 t3 t4 energy num_oc.
0   0   1   0   0   0   0   0  1  0  0  0  0  0  0  1  0  0  -26.0       1
['BINARY', 1 rows, 1 samples, 17 variables]


### Timings

In [221]:
print('Construting QUBO: {:.3f}'.format(qubo_time))
print('Construting BQM: {:.3f}'.format(bqm_time))
print('\nLocal deterministic solver: {:.1f}'.format(det_time))
print('Local heuristic solver: {:.1f}'.format(heur_time))
print('Quantum solver: {:.1f}'.format(qpu_time))

Construting QUBO: 0.374
Construting BQM: 0.257

Local deterministic solver: 251.4
Local heuristic solver: 1548.3
Quantum solver: 252.7


## More complex graph

In [24]:
seed = 42
vertices2 = 10
random.seed(seed)
G2 = nx.gnp_random_graph(vertices2, 0.30, seed, directed=True)
nx.set_edge_attributes(G2, {e: {'weight': random.randint(1, 10)} for e in G2.edges})

In [25]:
E2 = [] 
for e in G2.edges(data=True):
    E2.append((e[0],e[1],e[2]['weight']))
print('Number of edges:',len(E2))
print('Number of vertices:',vertices2)

max_count2 = 0
for e in E2:
    max_count2 += e[2]
print('Max count:',max_count2)

labels2 = {}
for i in range(vertices2):
    labels2[i]='s'+str(i)
    labels2[vertices2+i]='t'+str(i)   
for i,e in enumerate(E2):
    labels2[vertices2*2+i] = str(e[0]) + '-' + str(e[1])

Number of edges: 34
Number of vertices: 10
Max count: 166


### QUBO and BQM

In [26]:
t1 = time.time()
p = max_count2
Q2 = create_qubo(E2,vertices2,p)
qubo_time = (time.time()-t1) * 1000
print('Time used for construction Q (ms): {:.3f}'.format(qubo_time))

t1 = time.time()
bqm2 = dimod.BinaryQuadraticModel(Q2, 'BINARY')
bqm_time = (time.time()-t1) * 1000
bqm2 = bqm2.relabel_variables(labels2, inplace=False)
print('Time used for construction BQM (ms): {:.3f}'.format(bqm_time))

print('\nNumber of logical qubits needed:',Q2.shape[0])
print('Number of couplers needed:', len(bqm2.quadratic))

Time used for construction Q (ms): 1.100
Time used for construction BQM (ms): 0.345

Number of logical qubits needed: 54
Number of couplers needed: 438


In [27]:
t1 = time.time()
sampleset4 = dimod.ExactSolver().sample(bqm2)
det_time = (time.time()-t1)*1000
print('Time used (ms): {:.3f}\n'.format(det_time))
print(sampleset4.filter(lambda s: s.energy<0))

MemoryError: Unable to allocate 864. PiB for an array with shape (18014398509481984, 54) and data type int8

### Local heuristic solver

In [32]:
num_reads = 10000
t1 = time.time()
sampleset4 = SimulatedAnnealingSampler().sample(bqm2, num_reads=num_reads).aggregate()
heur_time = (time.time()-t1)*1000
print('Time used (ms): {:.3f}'.format(heur_time))
print('Lowest energy', sampleset4.first.energy)

Time used (ms): 7464.803
Lowest energy -165.0


In [33]:
check_result(G2,sampleset4,E2,vertices2)

Path: [8, 6, 2, 9, 0] (17): correct: [[8, 7, 3, 0]] (16)
99% ok


### Quantum solver

In [34]:
machine = DWaveSampler(solver={'chip_id': 'Advantage_system4.1'})
print('Chip:', machine.properties['chip_id'])
print('Qubits:', machine.properties['num_qubits'])

Chip: Advantage_system4.1
Qubits: 5760


In [39]:
num_reads = 3000
sampleset5 = EmbeddingComposite(machine).sample(bqm2, num_reads=num_reads)

In [40]:
qpu_time = sampleset5.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset5.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(qpu_time))
print('Lowest energy', sampleset5.first.energy)
print('Physical qubits used: {}'.format(qubits))

QPU time used (ms): 416.8
Lowest energy -154.0
Physical qubits used: 241


In [42]:
print('Number of logical qubits:',Q2.shape[0])
print('Number of couplers:', len(bqm2.quadratic))
qpu_time = sampleset5.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset5.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(qpu_time))
print('Physical qubits used: {}\n'.format(qubits))

Number of logical qubits: 54
Number of couplers: 438
QPU time used (ms): 416.8
Physical qubits used: 241



In [43]:
check_result(G2,sampleset5,E2,vertices2,verbose=False)
print('Chain breaks:',sampleset5.first.chain_break_fraction)

11% ok
Chain breaks: 0.0


In [44]:
print(sampleset5.truncate(10))

  0-2 0-3 0-4 0-8 1-0 1-2 1-4 1-5 1-8 2-1 2-6 2-9 3-0 ... t9 energy num_oc. ...
0   0   0   0   0   0   0   1   0   0   0   0   0   0 ...  1 -154.0       1 ...
1   1   0   0   0   0   0   0   0   0   0   0   1   1 ...  0 -153.0       1 ...
2   0   0   0   0   0   1   0   0   0   0   0   1   0 ...  0 -150.0       1 ...
3   0   0   0   0   0   1   0   0   0   0   0   1   0 ...  0 -148.0       1 ...
4   0   0   0   0   0   0   0   0   0   0   0   1   0 ...  0 -148.0       1 ...
5   0   0   1   0   0   0   0   0   0   0   0   0   0 ...  0 -148.0       1 ...
6   0   0   0   0   0   0   0   0   1   0   0   1   0 ...  0 -147.0       1 ...
7   0   0   1   0   1   0   0   0   0   0   0   0   0 ...  1 -147.0       1 ...
8   0   1   0   0   0   0   0   0   0   0   0   1   0 ...  0 -147.0       1 ...
9   0   1   0   0   1   0   0   0   0   0   0   1   0 ...  0 -146.0       1 ...
['BINARY', 10 rows, 10 samples, 54 variables]


In [232]:
chain_strength = p*2+1 #Default 1
num_reads = 1000
sampleset5 = EmbeddingComposite(machine).sample(bqm2, num_reads=num_reads, chain_strength=chain_strength)
qpu_time = sampleset5.info['timing']['qpu_access_time'] / 1000
qubits = sum(len(x) for x in sampleset5.info['embedding_context']['embedding'].values())
print('QPU time used (ms): {:.1f}'.format(qpu_time))
print('Lowest energy', sampleset5.first.energy)
print('Chain breaks:',sampleset5.first.chain_break_fraction)
print('Physical qubits used: {}\n'.format(qubits))
check_result(G2,sampleset5,E2,vertices2,verbose=False)

QPU time used (ms): 138.7
Lowest energy -154.0
Chain breaks: 0.0
Physical qubits used: 241

3% ok


### Timings

In [233]:
print('Construting QUBO (ms): {:.3f}'.format(qubo_time))
print('Construting BQM (ms): {:.3f}'.format(bqm_time))
print('\nLocal heuristic solver (ms): {:.1f}'.format(heur_time))
print('Quantum solver (ms): {:.1f}'.format(qpu_time))

Construting QUBO (ms): 1.170
Construting BQM (ms): 0.361

Local heuristic solver (ms): 7581.9
Quantum solver (ms): 138.7


## Test procedure #1

Test with 20 random graphs, with local heuristic solver

In [234]:
for seed in range(20):
    vertices = 10
    random.seed(seed)
    G = nx.gnp_random_graph(vertices, 0.30, seed, directed=True)
    nx.set_edge_attributes(G, {e: {'weight': random.randint(1, 10)} for e in G.edges})
    
    E = [] 
    for e in G.edges(data=True):
        E.append((e[0],e[1],e[2]['weight']))

    max_count = 0
    for e in E:
        max_count += e[2]

    labels = {}
    for i in range(vertices):
        labels[i]='s'+str(i)
        labels[vertices+i]='t'+str(i)   
    for i,e in enumerate(E):
        labels[vertices*2+i] = str(e[0]) + '-' + str(e[1])
        
    
    Q = create_qubo(E,vertices,max_count)
    bqm = dimod.BinaryQuadraticModel(Q, 'BINARY')
    bqm = bqm.relabel_variables(labels, inplace=False)

    num_reads = 20000
    sampleset = SimulatedAnnealingSampler().sample(bqm, num_reads=num_reads).aggregate()
    print('\nSeed:',seed)
    check_result(G,sampleset,E,vertices)


Seed: 0
100% ok

Seed: 1
100% ok

Seed: 2
100% ok

Seed: 3
100% ok

Seed: 4
100% ok

Seed: 5
100% ok

Seed: 6
100% ok

Seed: 7
100% ok

Seed: 8
100% ok

Seed: 9
100% ok

Seed: 10
100% ok

Seed: 11
100% ok

Seed: 12
100% ok

Seed: 13
100% ok

Seed: 14
100% ok

Seed: 15
100% ok

Seed: 16
100% ok

Seed: 17
100% ok

Seed: 18
100% ok

Seed: 19
100% ok
